## MultiTaskLearning retail data

In [1]:
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [3]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 5
train_batch_size = 64

2022-11-16 12:21:15 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [4]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.5
margin = 0.5

dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/retail_train/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_retailMultiTask-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [5]:
######### Read train data  ##########
train_samples_MultipleNegativesRankingLoss = []
train_samples_ConstrativeLoss = []

with open(os.path.join(dataset_path, "classification/train_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        train_samples_ConstrativeLoss.append(InputExample(texts=[row['product1'], 
                                                                 row['product2']], 
                                                          label=int(row['is_duplicate'])))
        if row['is_duplicate'] == '1':
            train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[row['product1'], 
                                                                                  row['product2']], 
                                                                           label=1))
            # if A is a duplicate of B, then B is a duplicate of A
            train_samples_MultipleNegativesRankingLoss.append(InputExample(texts=[row['product2'], 
                                                                                  row['product1']], 
                                                                           label=1))  

# Create data loader and loss for MultipleNegativesRankingLoss
train_dataloader_MultipleNegativesRankingLoss = DataLoader(train_samples_MultipleNegativesRankingLoss, 
                                                           shuffle=True, 
                                                           batch_size=train_batch_size)
train_loss_MultipleNegativesRankingLoss = losses.MultipleNegativesRankingLoss(model)


# Create data loader and loss for OnlineContrastiveLoss
train_dataloader_ConstrativeLoss = DataLoader(train_samples_ConstrativeLoss, 
                                              shuffle=True, 
                                              batch_size=train_batch_size)
train_loss_ConstrativeLoss = losses.OnlineContrastiveLoss(model=model, 
                                                          distance_metric=distance_metric, 
                                                          margin=margin)

In [6]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Products pair classification,
# Duplicate Products Mining, and Duplicate Products Information Retrieval
evaluators = []

###### Classification ######
# Given (product1, product2), is this a duplicate or not?
# The evaluator will compute the embeddings for both products and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []
with open(os.path.join(dataset_path, "classification/dev_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences1.append(row['product1'])
        dev_sentences2.append(row['product2'])
        dev_labels.append(int(row['is_duplicate']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [7]:
###### Duplicate Products Mining ######
# Given a large corpus of products, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
max_dev_samples = 100_000
dev_sentences = {}
dev_duplicates = []
with open(os.path.join(dataset_path, "duplicate-mining/dev_corpus.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences[row['qid']] = row['product']

        if len(dev_sentences) >= max_dev_samples:
            break

with open(os.path.join(dataset_path, "duplicate-mining/dev_duplicates.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['qid1'] in dev_sentences and row['qid2'] in dev_sentences:
            dev_duplicates.append([row['qid1'], row['qid2']])


# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, 
                                                                   dev_duplicates, 
                                                                   name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [8]:
###### Duplicate Products Information Retrieval ######
# Given a product and a large corpus of thousands products, 
# find the most relevant (i.e. duplicate) product in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
max_corpus_size = 100_000

ir_queries = {}             #Our queries (qid => question)
ir_needed_qids = set()      #QIDs we need in the corpus
ir_corpus = {}              #Our corpus (qid => question)
ir_relevant_docs = {}       #Mapping of relevant documents for a given query 
                            #(qid => set([relevant_question_ids])

with open(os.path.join(dataset_path, 'information-retrieval/dev-queries.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, query, duplicate_ids = line.strip().split('\t')
        duplicate_ids = duplicate_ids.split(',')
        ir_queries[qid] = query
        ir_relevant_docs[qid] = set(duplicate_ids)

        for qid in duplicate_ids:
            ir_needed_qids.add(qid)

In [9]:
# First get all needed relevant documents 
# (i.e., we must ensure, that the relevant products are actually in the corpus
distraction_products = {}
with open(os.path.join(dataset_path, 'information-retrieval/corpus.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, product = line.strip().split('\t')
        if qid in ir_needed_qids:
            ir_corpus[qid] = product
        else:
            distraction_products[qid] = product

# Now, also add some irrelevant products to fill our corpus
other_qid_list = list(distraction_products.keys())
random.shuffle(other_qid_list)

for qid in other_qid_list[0:max(0, max_corpus_size-len(ir_corpus))]:
    ir_corpus[qid] = distraction_products[qid]

In [12]:
#Given queries, a corpus and a mapping with relevant documents, 
# the InformationRetrievalEvaluator computes different IR
# metrices. For our use case MRR@k and Accuracy@k are relevant.
ir_evaluator = evaluation.InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)

evaluators.append(ir_evaluator)

In [13]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three 
# evaluators in a sequential order. We optimize the model with respect
# to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, 
                                               main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

2022-11-16 13:58:11 - Evaluate model without training
2022-11-16 13:58:11 - Binary Accuracy Evaluation of the model on  dataset in epoch 0 after 0 steps:
2022-11-16 13:58:20 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8767)
2022-11-16 13:58:20 - F1 with Cosine-Similarity:                 55.65	(Threshold: 0.7286)
2022-11-16 13:58:20 - Precision with Cosine-Similarity:          46.94
2022-11-16 13:58:20 - Recall with Cosine-Similarity:             68.32
2022-11-16 13:58:20 - Average Precision with Cosine-Similarity:  47.39

2022-11-16 13:58:20 - Accuracy with Manhattan-Distance:           79.27	(Threshold: 7.6162)
2022-11-16 13:58:20 - F1 with Manhattan-Distance:                 56.22	(Threshold: 11.4881)
2022-11-16 13:58:20 - Precision with Manhattan-Distance:          47.30
2022-11-16 13:58:20 - Recall with Manhattan-Distance:             69.31
2022-11-16 13:58:20 - Average Precision with Manhattan-Distance:  47.65

2022-11-16 13:58:20 - Accuracy with Euclidean-Di

In [14]:
# Train the model
model.fit(train_objectives=[(train_dataloader_MultipleNegativesRankingLoss, 
                             train_loss_MultipleNegativesRankingLoss), 
                            (train_dataloader_ConstrativeLoss, 
                             train_loss_ConstrativeLoss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
          )

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 13:59:22 - Binary Accuracy Evaluation of the model on  dataset after epoch 0:
2022-11-16 13:59:22 - Accuracy with Cosine-Similarity:           79.27	(Threshold: 0.8824)
2022-11-16 13:59:22 - F1 with Cosine-Similarity:                 55.87	(Threshold: 0.7316)
2022-11-16 13:59:22 - Precision with Cosine-Similarity:          47.26
2022-11-16 13:59:22 - Recall with Cosine-Similarity:             68.32
2022-11-16 13:59:22 - Average Precision with Cosine-Similarity:  47.66

2022-11-16 13:59:22 - Accuracy with Manhattan-Distance:           79.50	(Threshold: 7.6683)
2022-11-16 13:59:22 - F1 with Manhattan-Distance:                 56.45	(Threshold: 11.4677)
2022-11-16 13:59:22 - Precision with Manhattan-Distance:          47.62
2022-11-16 13:59:22 - Recall with Manhattan-Distance:             69.31
2022-11-16 13:59:22 - Average Precision with Manhattan-Distance:  47.88

2022-11-16 13:59:22 - Accuracy with Euclidean-Distance:           79.27	(Threshold: 0.4849)
2022-11-16 13:59:22 -

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 13:59:37 - Binary Accuracy Evaluation of the model on  dataset after epoch 1:
2022-11-16 13:59:37 - Accuracy with Cosine-Similarity:           79.50	(Threshold: 0.8765)
2022-11-16 13:59:37 - F1 with Cosine-Similarity:                 56.34	(Threshold: 0.6877)
2022-11-16 13:59:37 - Precision with Cosine-Similarity:          43.72
2022-11-16 13:59:37 - Recall with Cosine-Similarity:             79.21
2022-11-16 13:59:37 - Average Precision with Cosine-Similarity:  48.47

2022-11-16 13:59:37 - Accuracy with Manhattan-Distance:           79.73	(Threshold: 8.0373)
2022-11-16 13:59:37 - F1 with Manhattan-Distance:                 56.78	(Threshold: 11.1804)
2022-11-16 13:59:37 - Precision with Manhattan-Distance:          49.63
2022-11-16 13:59:37 - Recall with Manhattan-Distance:             66.34
2022-11-16 13:59:37 - Average Precision with Manhattan-Distance:  48.84

2022-11-16 13:59:37 - Accuracy with Euclidean-Distance:           79.50	(Threshold: 0.4969)
2022-11-16 13:59:37 -

...

2022-11-16 13:59:51 - Binary Accuracy Evaluation of the model on  dataset after epoch 2:
2022-11-16 13:59:51 - Accuracy with Cosine-Similarity:           80.18	(Threshold: 0.8360)
2022-11-16 13:59:51 - F1 with Cosine-Similarity:                 57.55	(Threshold: 0.6868)
2022-11-16 13:59:51 - Precision with Cosine-Similarity:          45.20
2022-11-16 13:59:51 - Recall with Cosine-Similarity:             79.21
2022-11-16 13:59:51 - Average Precision with Cosine-Similarity:  49.73

2022-11-16 13:59:51 - Accuracy with Manhattan-Distance:           80.41	(Threshold: 8.8687)
2022-11-16 13:59:51 - F1 with Manhattan-Distance:                 57.26	(Threshold: 11.1972)
2022-11-16 13:59:51 - Precision with Manhattan-Distance:          50.38
2022-11-16 13:59:51 - Recall with Manhattan-Distance:             66.34
2022-11-16 13:59:51 - Average Precision with Manhattan-Distance:  49.92

2022-11-16 13:59:51 - Accuracy with Euclidean-Distance:           80.18	(Threshold: 0.5728)
2022-11-16 13:59:51 -

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 14:00:05 - Binary Accuracy Evaluation of the model on  dataset after epoch 3:
2022-11-16 14:00:05 - Accuracy with Cosine-Similarity:           80.64	(Threshold: 0.8484)
2022-11-16 14:00:05 - F1 with Cosine-Similarity:                 58.77	(Threshold: 0.7431)
2022-11-16 14:00:05 - Precision with Cosine-Similarity:          52.76
2022-11-16 14:00:05 - Recall with Cosine-Similarity:             66.34
2022-11-16 14:00:05 - Average Precision with Cosine-Similarity:  50.88

2022-11-16 14:00:05 - Accuracy with Manhattan-Distance:           80.64	(Threshold: 8.4629)
2022-11-16 14:00:05 - F1 with Manhattan-Distance:                 58.77	(Threshold: 11.0927)
2022-11-16 14:00:05 - Precision with Manhattan-Distance:          52.76
2022-11-16 14:00:05 - Recall with Manhattan-Distance:             66.34
2022-11-16 14:00:05 - Average Precision with Manhattan-Distance:  51.05

2022-11-16 14:00:05 - Accuracy with Euclidean-Distance:           80.64	(Threshold: 0.5506)
2022-11-16 14:00:05 -

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-16 14:00:19 - Binary Accuracy Evaluation of the model on  dataset after epoch 4:
2022-11-16 14:00:19 - Accuracy with Cosine-Similarity:           80.41	(Threshold: 0.8518)
2022-11-16 14:00:19 - F1 with Cosine-Similarity:                 61.14	(Threshold: 0.7434)
2022-11-16 14:00:19 - Precision with Cosine-Similarity:          54.69
2022-11-16 14:00:19 - Recall with Cosine-Similarity:             69.31
2022-11-16 14:00:19 - Average Precision with Cosine-Similarity:  52.08

2022-11-16 14:00:19 - Accuracy with Manhattan-Distance:           80.41	(Threshold: 8.2067)
2022-11-16 14:00:19 - F1 with Manhattan-Distance:                 60.76	(Threshold: 11.3684)
2022-11-16 14:00:19 - Precision with Manhattan-Distance:          52.94
2022-11-16 14:00:19 - Recall with Manhattan-Distance:             71.29
2022-11-16 14:00:19 - Average Precision with Manhattan-Distance:  52.22

2022-11-16 14:00:19 - Accuracy with Euclidean-Distance:           80.41	(Threshold: 0.5445)
2022-11-16 14:00:19 -